In [1]:
import json
import os

In [2]:
data = []
with open('prepared-wikipedia-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l['text'])

In [3]:
len(data)

97963

In [4]:
# !rm -rf generate-wikipedia-qa-llama3
!mkdir generate-wikipedia-qa-llama3

mkdir: cannot create directory ‘generate-wikipedia-qa-llama3’: File exists


In [5]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

Generate list of QA choice in malay, return in JSON [{'question', 'A', 'B', 'C', 'D', 'answer'}]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

97963

In [6]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'generate-wikipedia-qa-llama3/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [7]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [8]:
urls = [(q, no) for no, q in enumerate(results)]

In [9]:
answer(*urls[10005])

In [10]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 98%|██████████████████████████████████████████████████████████████████████▌ | 96022/97963 [31:13:03<25:29,  1.27it/s]

In [4]:
data = []
with open('prepared-wikipedia-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l)

In [5]:
suppose = {'question', 'A', 'B', 'C', 'D', 'answer'}

In [6]:
from tqdm import tqdm

with open('generated-wikipedia-qa-llama3.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'generate-wikipedia-qa-llama3/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                selected = []
                for d__ in d_.split('```'):
                    try:
                        try:
                            selected.append(json.loads(d__))
                        except:
                            selected.append(eval(d__))
                    except:
                        pass
                if not len(selected):
                    continue
                qa = selected[0]
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('A' in q and isinstance(q['A'], str)):
                            continue
                        if not ('B' in q and isinstance(q['B'], str)):
                            continue
                        if not ('C' in q and isinstance(q['C'], str)):
                            continue
                        if not ('D' in q and isinstance(q['D'], str)):
                            continue
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
            
        if not len(questions):
            continue
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

100%|█████████████████████████████████████████████████████████████████████████| 97963/97963 [00:15<00:00, 6454.26it/s]


In [7]:
!wc -l generated-wikipedia-qa.jsonl

96178 generated-wikipedia-qa.jsonl


In [8]:
!ls -lh generated-wikipedia-qa.jsonl

-rw-rw-r-- 1 husein husein 433M Mei   9 16:05 generated-wikipedia-qa.jsonl


In [9]:
!tail -n 1 generated-wikipedia-qa.jsonl

{"url": "https://ms.wikipedia.org/wiki?curid=1163129", "text": "DeadSquad adalah kugiran death metal yang berasal dari Jakarta. Kugiran ini awalnya dibentuk pada tahun 2006 oleh Stevie Item (Step Forward), Ricky Siahaan (Seringai), Bonny (Tengkorak) dan Andyan Gorust (Siksakubur) sebagai \"Project Band\". Pada awalnya, kugiran ini dibentuk hanya sebagai bentuk \"tribute\" dengan mempersembahkan lagu-lagu thrash metal seperti Slayer, Anthrax, dan Sepultura. Saat ini DeadSquad telah berkembang dan terbukti tidak hanya sekadar \"Band Project\" tetapi menjadi salah satu adikugiran terbaik yang kewujudannya berpengaruh besar di adegan lagu metal Indonesia.", "questions": [{"question": "Siapakah yang membentuk kugiran DeadSquad?", "A": "Stevie Item", "B": "Ricky Siahaan", "C": "Bonny", "D": "Andyan Gorust", "answer": "A"}, {"question": "Apakah genre muzik yang dimainkan oleh DeadSquad?", "A": "Thrash metal", "B": "Death metal", "C": "Black metal", "D": "Reggae", "answer": "B"}, {"question": 

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='generated-wikipedia-qa-llama3.jsonl',
    path_in_repo='generated-wikipedia-qa-llama3.jsonl',
    repo_id='mesolitica/llama-3-70b-qa',
    repo_type='dataset',
)

generated-wikipedia-qa-llama3.jsonl:   0%|          | 0.00/340M [00:00<?, ?B/s]